In [16]:
import soundfile as sf
import numpy as np
import glob
import random

In [9]:
male_path = '/home/jhkim21/Data/raw/sitec/SiTEC_Dict01_reading_sentence/SD01-Dict01-Ver1.2_1-2/data/male/**/*'
female_path = '/home/jhkim21/Data/raw/sitec/SiTEC_Dict01_reading_sentence/SD01-Dict01-Ver1.2_2-2/data/female/**/*'

In [10]:
male_folder = glob.glob(male_path)
female_folder = glob.glob(female_path)

In [15]:
male_folder = sorted(male_folder, key=lambda a: int(a.split('/')[-1][-3:]))
female_folder = sorted(female_folder, key=lambda a: int(a.split('/')[-1][-3:]))

In [ ]:
def concat(wav1, wav2):
    len1 = wav1.shape()
    len2 = wav2.shape()
    
    trim_len1 = random.randint(len1//2, len1)
    trim_len2 = random.randint(len2//2, len2)
    
    idx1 = random.randint(0, len1 - trim_len1)
    idx2 = random.randint(0, len2 - trim_len2)
    trim_wav1 = wav1[idx1:idx1+trim_len1]
    trim_wav2 = wav2[idx2:idx2+trim_len2]
    
    clean = np.concatenate((trim_wav1, trim_wav2))
    
    if random.randint(0, 1):
        max1 = max(trim_wav1)
        gain = random.uniform(0.01, (1/max1))
        trim_wav1 *= gain
    else:
        max2 = max(trim_wav2)
        gain = random.uniform(0.01, (1/max2))
        trim_wav2 *= gain
    
    noisy = np.concatenate((trim_wav1, trim_wav2))
    
    return clean, noisy